### SPLIT AFRICA DATASET

I re-ran this notebook until the split was reasonably even between train/validation/test. In a perfect split the `low-med-high rfracs` would all equal 1 and the `block split fracs` would equal the respective `group fracs`.  I am not too concerned with less validation data, but prefer that `water-tag=low` be well represented in both test and validation.

---

In [1]:
import pandas as pd
from random import sample

In [2]:
DSET='surface-water.africa'
NO_DATA_MAX=0.05

In [3]:
df=pd.read_csv(f'{DSET}.csv')
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=['group_id']).shape[0])
df=df[df.no_data<=NO_DATA_MAX]
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=['group_id']).shape[0])
df.sample(3)

4517 656 4405
3956 615 3864


,gsw_path,s1_path,block_id,group_id,water,not_water,no_data,dataset
3437,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_28_-31,group_28.71_-31.47,0.231907,0.757656,0.010437,africa_mtn
2910,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_30_-26,group_30.63_-26.52,0.063629,0.936371,0.000000,africa_mtn
2723,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_29_-29,group_29.83_-29.33,0.008507,0.990784,0.000710,africa_mtn


---

In [4]:
def water_counter(df,quantiles=[0.15,0.85],labels=['low','medium','high']):
    wqs=[0]+[df.water.quantile(q) for q in quantiles]+[1+1e-8]
    dfs=[]
    for i,l in enumerate(labels):
        _df=df.copy()[(df.water>=wqs[i])&(df.water<wqs[i+1])]
        _df['water_tag']=l
        dfs.append(_df)
    return pd.concat(dfs).sample(frac=1)


def split_groups(blocks,valid_frac=0.2,test_frac=0.1,hold_out_frac=0.2,relative_frac=True):
    if relative_frac:
        valid_frac=(1-hold_out_frac)*valid_frac
        test_frac=(1-hold_out_frac)*test_frac
    nb_blocks=len(blocks)
    nb_valid=int(valid_frac*nb_blocks)    
    nb_test=int(test_frac*nb_blocks)
    nb_hold_out=int(hold_out_frac*nb_blocks)
    blocks=sample(blocks,nb_blocks)
    valid=blocks[:nb_valid]
    test=blocks[nb_valid:nb_valid+nb_test]
    hold_out=blocks[nb_valid+nb_test:nb_valid+nb_test+nb_hold_out]
    train=blocks[nb_valid+nb_test+nb_hold_out:]
    rtotal=len(train)+len(valid)+len(test)
    print('block relative split:',len(train)/rtotal,len(valid)/rtotal,len(test)/rtotal,len(hold_out)/nb_blocks)
    print('block split fracs:',len(train)/nb_blocks,len(valid)/nb_blocks,len(test)/nb_blocks,len(hold_out)/nb_blocks)
    return train, valid, test, hold_out


def sub_df(typ,df,blocks,cnt):
    
    sdf=df.copy()[df.block_id.isin(blocks)]
    sdf['data_split']=typ
    
    sgcnt=sdf.drop_duplicates(subset=['group_id']).shape[0]
    frac=sgcnt/cnt
    
    ltcnt=df[df.water_tag=='low'].shape[0]*frac
    mtcnt=df[df.water_tag=='medium'].shape[0]*frac
    htcnt=df[df.water_tag=='high'].shape[0]*frac
    
    sltcnt=sdf[sdf.water_tag=='low'].shape[0]
    smtcnt=sdf[sdf.water_tag=='medium'].shape[0]
    shtcnt=sdf[sdf.water_tag=='high'].shape[0]
    print()
    print(f'{typ} group:',frac,f'({sgcnt}/{cnt})  [{int(ltcnt)},{int(mtcnt)},{int(htcnt)}]')
    print('* low-med-high rfracs:',sltcnt/ltcnt,smtcnt/mtcnt,shtcnt/htcnt)
    return sdf
    
    
def df_splitter(df,train, valid, test, hold_out):
    cnt=df.drop_duplicates(subset=['group_id']).shape[0]
    tdf=sub_df('train',df,train,cnt)    
    vdf=sub_df('valid',df,valid,cnt)    
    sdf=sub_df('test',df,test,cnt)    
    hdf=sub_df('hold_out',df,hold_out,cnt)
    df=pd.concat([tdf, vdf, sdf, hdf])
    return df

---

In [5]:
df=water_counter(df)

In [6]:
blocks=list(df.block_id.unique())

In [7]:
print('-'*85)
train, valid, test, hold_out=split_groups(blocks)
print('-'*85)
df=df_splitter(df, train, valid, test, hold_out)

-------------------------------------------------------------------------------------
block relative split: 0.7012195121951219 0.1991869918699187 0.09959349593495935 0.2
block split fracs: 0.5609756097560976 0.15934959349593497 0.07967479674796749 0.2
-------------------------------------------------------------------------------------

train group: 0.609472049689441 (2355/3864)  [362,1687,362]
* low-med-high rfracs: 1.0606961333075984 0.9940613379477927 0.9778292478929421

valid group: 0.1379399585921325 (533/3864)  [81,381,81]
* low-med-high rfracs: 1.0007770007770007 1.0057152772505937 0.9275494153542934

test group: 0.08462732919254658 (327/3864)  [50,234,50]
* low-med-high rfracs: 0.9349766780959442 0.9349048098849234 1.3527322151175363

hold_out group: 0.1679606625258799 (649/3864)  [99,464,99]
* low-med-high rfracs: 0.8118784143437457 1.049653980779679 0.9622262688518467


---

In [8]:
df.sample(3)

,gsw_path,s1_path,block_id,group_id,water,not_water,no_data,dataset,water_tag,data_split
4320,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_27_-33,group_27.4_-33.33,0.106003,0.852798,0.041199,africa_mtn,medium,train
2071,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_-14_11,group_-14.93_11.27,0.033314,0.966686,0.000000,africa,medium,train
3515,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_-14_11,group_-14.83_11.74,0.039894,0.960106,0.000000,africa_mtn,medium,train


In [9]:
file_name=f'{DSET}.split.csv'
df.to_csv(file_name,index=False)
print(f'UPLOAD: gsutil cp {file_name} gs://surface-water-public/data/v1/datasets')

UPLOAD: gsutil cp surface-water.africa.split.csv gs://surface-water-public/data/v1/datasets


---

In [16]:
# !gsutil cp surface-water.africa.split.csv gs://surface-water-public/data/v1/datasets

---

##### WATER/NOT RATIO

In [11]:
_df=df[df.data_split.isin(['train','valid'])]

In [12]:
_df.not_water.mean()/_df.water.mean()

12.668450830567274

In [13]:
_df.drop_duplicates(subset=['group_id'])[['water','not_water']].describe()

,water,not_water
count,2888.000000,2888.000000
mean,0.072785,0.922525
std,0.082871,0.084227
min,0.005032,0.411617
25%,0.017656,0.900648
50%,0.036377,0.958939
75%,0.092906,0.979426
max,0.588383,0.994911
